## Dask dependency management plugins

Dask’s plugin system enables you to run custom Python code for certain events. You can use plugins that are specific to schedulers, workers, or nannies. A worker plugin, for example, allows you to run custom Python code on all your workers at certain event in the worker’s lifecycle (e.g. when the worker process is started). Let's check dependency management plugins allowing you to install packages on workers:

In [4]:
from dask.distributed import Client

client = Client("tls://localhost:8786")
client

Connection method: Direct,
Dashboard: /user/oksana.shadura@cern.ch/proxy/35575/status,
Comm: tls://192.168.121.81:8786,Workers: 0
Dashboard: /user/oksana.shadura@cern.ch/proxy/35575/status,Total threads: 0
Started: 16 minutes ago,Total memory: 0 B


In [18]:
from dask.distributed import PipInstall

plugin = PipInstall(packages=["hepconvert"], pip_options=["--upgrade"])

client.register_plugin(plugin)

Or we can simply execute custom function on worker:

In [6]:
def worker_setup(dask_worker):
    import os
    #install_root_packages_cmd = "mamba install -y -c conda-forge root"
    install_root_packages_cmd = "mamba install -y -c conda-forge correctionlib"
    os.system(install_root_packages_cmd)

In [7]:
client.register_worker_callbacks(worker_setup)

{}

Or to enable CMSSW environmnet:

In [8]:
 def worker_cmssw_setup(dask_worker):
    import os
    install_cmssw_packages_cmd = "source /cvmfs/cms.cern.ch/cmsset_default.sh; cd /cvmfs/cms.cern.ch/${SCRAM_ARCH}/cms/cmssw/CMSSW_12_6_5; cmsenv"
    os.system(install_cmssw_packages_cmd)

In [9]:
client.register_worker_callbacks(worker_cmssw_setup)

{}

Or enable environment variable:

In [10]:
def set_env(dask_worker):
        import pathlib, os
        path = str(pathlib.Path(dask_worker.local_directory))
        os.environ["HOME_DIR"] = path

In [11]:
client.register_worker_callbacks(set_env)

{}

You can simply create your plugins:

In [12]:
from dask.distributed import WorkerPlugin
class ErrorLogger(WorkerPlugin):
    def __init__(self, logger):
        self.logger = logger

    def setup(self, worker):
        self.worker = worker

    def transition(self, key, start, finish, *args, **kwargs):
        if finish == 'error':
            ts = self.worker.tasks[key]
            exc_info = (type(ts.exception), ts.exception, ts.traceback)
            self.logger.error(
                "Error during computation of '%s'.", key,
                exc_info=exc_info
            )

In [13]:
import logging
plugin = ErrorLogger(logging)
client.register_plugin(plugin) 

{}

Or you can upload your file using plugin `UploadFile` (`UploadDirectory` doesnt work for now on coffea-casa, we are fixing it):

In [16]:
from distributed.diagnostics.plugin import UploadFile

client.register_plugin(UploadFile("upload_directory/bar.py"))  

{}